## Import Packages

In [1]:
!pip install gurobipy>=9.5.1

zsh:1: 9.5.1 not found


In [2]:
import gurobipy as gp
from gurobipy import GRB as GRB
import sys
sys.path.insert(0, "../utils")

import column_generation
from unavailability import optimal_group1_unavailability

In [3]:
path_prefix = "../data/combined_model/group1_results.txt"

## Column Generation

In [4]:
generic_columns = column_generation.generic_column_generation(5)
group_feasible_combinations = column_generation.group_column_generation(1, optimal_group1_unavailability, generic_columns)

Total possible combinations: 1048576
Total feasible combinations: 45046
Group 1 Unavailability Mapping: {1: [15], 2: [7, 11, 15], 3: [15], 4: [15], 5: [0, 1, 2, 3, 4, 5, 6, 9, 10, 12, 13, 14, 15, 16, 19], 6: [15], 7: [11, 13], 8: [8, 11, 13, 18], 9: [11, 13], 10: [4, 6, 8, 11, 13, 16], 11: [4, 6, 7, 8, 11, 13, 16], 12: [4, 6, 7, 8, 11, 13, 16], 13: [4, 6, 7, 8, 16], 14: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 12, 13, 14, 16, 19], 15: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 12, 13, 14, 16, 19], 16: [4, 7, 16], 17: [4, 7, 8, 11, 16, 18], 18: [4, 7, 8, 16, 18], 19: [], 20: [0, 1, 2, 3, 4, 5, 6, 9, 10, 12, 13, 14, 16, 19], 21: [], 22: [], 23: [8, 18], 24: [], 25: [], 26: [], 27: [], 28: [], 29: [], 30: [], 31: [], 32: [], 33: [], 34: [], 35: [15, 19], 36: [7, 11, 15, 19], 37: [15, 19], 38: [0, 5, 15, 19], 39: [0, 1, 2, 3, 4, 5, 6, 9, 10, 12, 13, 14, 15, 16, 19], 40: [0, 5, 15, 19], 41: [0, 3, 5, 14], 42: [0, 3, 5, 8, 14, 18], 43: [0, 3, 5, 14], 44: [3, 14], 45: [3, 7, 11, 14], 46: [3, 7, 11, 14], 47

## Optimization Model

In [5]:
# Setup Gurobi key
# Create environment with WLS license
e = gp.Env(empty=True)
e.setParam('WLSACCESSID', '453f2da9-b8f2-448f-9f06-48081a9c3dc9')
e.setParam('WLSSECRET', 'd5675c81-7058-45f9-b6f2-30ca3855ed22')
e.setParam('LICENSEID', 868440)
e.start()

# Create the model within the Gurobi environment. This environment will be passed in as a parameter into all subsequent models
model = gp.Model(env=e)
model.setParam('Seed', 435)

Set parameter WLSAccessID
Set parameter WLSSecret
Set parameter LicenseID to value 868440
Academic license 868440 - for non-commercial use only - registered to es___@uwaterloo.ca
Set parameter Seed to value 435


In [6]:
# Sets
D = range(1,6)
T = range(1,171)
R = range(1,170)
T_d = {d : (T[(d-1)*len(T)//len(D):(d)*len(T)//len(D)])for d in D}
C = range(1,6)
M = range(6,21)
I = (*C,*M)
H_t = {t : range(1,len(group_feasible_combinations[t-1])) for t in T}

In [7]:
# penalties
# separated to make them easier to read
# taken from this graph: https://www.desmos.com/calculator/g5rhs6n2ke
penalties = [(0,0), (4,4), (8, 11.31), (12, 20.78), (16, 32), (34, 99.13)]
Q = range(1,len(penalties)+1)

In [8]:
# Parameters

# need to sort this one out with column generation

a = group_feasible_combinations
f = [[sum(a[t-1][h-1]) // 6 for h in H_t[t]] for t in T]
n = 8 # Need to inpute
u = {q : penalties[q-1][1] for q in Q}
v = {q : penalties[q-1][0] for q in Q}
w = .001 # Need to inpute
g = .01

In [9]:
indices = [(t, h) for t in T for h in H_t[t]]

In [10]:
# Decision Varbiables
alpha = model.addVars(indices, vtype=GRB.BINARY, name="alpha")
y = model.addVars(I, D, Q, lb=0.0, ub=1.0, vtype=GRB.CONTINUOUS, name = "y")
z = model.addVars(I, R, vtype=GRB.BINARY, name = "z")

In [11]:
for i in I:
    for d in D:
        model.addSOS(GRB.SOS_TYPE2, [y[i,d,q] for q in Q])

In [12]:
# Objective Value
model.setObjective(sum(f[t-1][h-1]*alpha[t,h] for t in T for h in H_t[t]) - w * sum(u[q] * y[i,d,q] for i in I for d in D for q in Q) - g * sum(z[i, r] for i in I for r in R),sense=GRB.MAXIMIZE)

In [13]:
model.addConstrs(sum(a[r-1][h-1][i-1]*alpha[r,h] for h in H_t[r]) + sum(a[r][h-1][i-1]*alpha[r+1,h] for h in H_t[r+1]) - 1 <= z[i,r] for i in I for r in R)

{(1, 1): <gurobi.Constr *Awaiting Model Update*>,
 (1, 2): <gurobi.Constr *Awaiting Model Update*>,
 (1, 3): <gurobi.Constr *Awaiting Model Update*>,
 (1, 4): <gurobi.Constr *Awaiting Model Update*>,
 (1, 5): <gurobi.Constr *Awaiting Model Update*>,
 (1, 6): <gurobi.Constr *Awaiting Model Update*>,
 (1, 7): <gurobi.Constr *Awaiting Model Update*>,
 (1, 8): <gurobi.Constr *Awaiting Model Update*>,
 (1, 9): <gurobi.Constr *Awaiting Model Update*>,
 (1, 10): <gurobi.Constr *Awaiting Model Update*>,
 (1, 11): <gurobi.Constr *Awaiting Model Update*>,
 (1, 12): <gurobi.Constr *Awaiting Model Update*>,
 (1, 13): <gurobi.Constr *Awaiting Model Update*>,
 (1, 14): <gurobi.Constr *Awaiting Model Update*>,
 (1, 15): <gurobi.Constr *Awaiting Model Update*>,
 (1, 16): <gurobi.Constr *Awaiting Model Update*>,
 (1, 17): <gurobi.Constr *Awaiting Model Update*>,
 (1, 18): <gurobi.Constr *Awaiting Model Update*>,
 (1, 19): <gurobi.Constr *Awaiting Model Update*>,
 (1, 20): <gurobi.Constr *Awaiting Model

In [14]:
# Constraints
model.addConstrs(sum(alpha[t,h] for h in H_t[t]) <= 1 for t in T)
model.addConstrs(sum(a[t-1][h-1][i-1]*alpha[t,h] for t in T_d[d] for h in H_t[t]) <= n for i in I for d in D)
model.addConstrs(sum(a[t-1][h-1][i-1]*alpha[t,h] for t in T_d[d] for h in H_t[t]) == sum(v[q]*y[i,d,q] for q in Q) for i in I for d in D)
model.addConstrs(sum(y[i,d,q] for q in Q) == 1 for i in I for d in D)

{(1, 1): <gurobi.Constr *Awaiting Model Update*>,
 (1, 2): <gurobi.Constr *Awaiting Model Update*>,
 (1, 3): <gurobi.Constr *Awaiting Model Update*>,
 (1, 4): <gurobi.Constr *Awaiting Model Update*>,
 (1, 5): <gurobi.Constr *Awaiting Model Update*>,
 (2, 1): <gurobi.Constr *Awaiting Model Update*>,
 (2, 2): <gurobi.Constr *Awaiting Model Update*>,
 (2, 3): <gurobi.Constr *Awaiting Model Update*>,
 (2, 4): <gurobi.Constr *Awaiting Model Update*>,
 (2, 5): <gurobi.Constr *Awaiting Model Update*>,
 (3, 1): <gurobi.Constr *Awaiting Model Update*>,
 (3, 2): <gurobi.Constr *Awaiting Model Update*>,
 (3, 3): <gurobi.Constr *Awaiting Model Update*>,
 (3, 4): <gurobi.Constr *Awaiting Model Update*>,
 (3, 5): <gurobi.Constr *Awaiting Model Update*>,
 (4, 1): <gurobi.Constr *Awaiting Model Update*>,
 (4, 2): <gurobi.Constr *Awaiting Model Update*>,
 (4, 3): <gurobi.Constr *Awaiting Model Update*>,
 (4, 4): <gurobi.Constr *Awaiting Model Update*>,
 (4, 5): <gurobi.Constr *Awaiting Model Update*>,


In [15]:
model.optimize()

Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (mac64[arm] - Darwin 23.4.0 23E224)

CPU model: Apple M2
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Academic license 868440 - for non-commercial use only - registered to es___@uwaterloo.ca
Optimize a model with 3850 rows, 3877446 columns and 153938324 nonzeros
Model fingerprint: 0x7a08014d
Model has 100 SOS constraints
Variable types: 600 continuous, 3876846 integer (3876846 binary)
Coefficient statistics:
  Matrix range     [1e+00, 3e+01]
  Objective range  [4e-03, 2e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 8e+00]
Presolve removed 20 rows and 0 columns (presolve time = 6s) ...
Presolve removed 20 rows and 150 columns (presolve time = 16s) ...
Presolve removed 324 rows and 150 columns (presolve time = 23s) ...
Presolve removed 324 rows and 150 columns (presolve time = 26s) ...
Presolve removed 324 rows and 150 columns (presolve time = 34s) ...
Presolve removed 324 rows and 454 co

: 

In [ ]:
with open(path_prefix, "w") as file:
  for t in T:
    for h in H_t[t]:
      if alpha[t,h].X == 1:
        file.write(f"{t} {a[t-1][h-1]}\n")
        print(t, a[t-1][h-1])


In [ ]:
print(sum(alpha[t,h].X * sum(a[t-1][h-1][i-1] for i in I) for t in T for h in H_t[t]) // 6)